In [4]:
%load_ext autoreload
%aimport os, pandas, numpy, pickle
%autoreload 2

In [5]:
import sys
import os
import pickle
import argparse
import pandas as pd
from sklearn.ensemble.forest import ExtraTreesRegressor

from fqi.et_tuning import run_tuning
from fqi.reward_function import *
from fqi.sars_creator import to_SARS
from fqi.utils import *

from trlib.policies.valuebased import EpsilonGreedy, Softmax
from trlib.policies.qfunction import ZeroQ
from trlib.algorithms.reinforcement.fqi_driver import FQIDriver, DoubleFQIDriver
from trlib.environments.trackEnv import TrackEnv
from trlib.utilities.ActionDispatcher import *

from fqi.fqi_evaluate import run_evaluation

In [6]:
simulations = pd.read_csv('./trajectory/dataset_70_laps.csv',
                              dtype={'isReference': bool, 'is_partial':bool})

#simulations = simulations[(simulations['NLap'] == 1) | (simulations['NLap'] == 9)]

ref_tr = pd.read_csv('./trajectory/ref_traj.csv')
ref_tr.columns = ['time', 'Acceleration_x', 'Acceleration_y', 'speed_x', 'speed_y',
       'xCarWorld', 'yCarWorld', 'alpha_step']

In [7]:
file_name = 'model_file/penalty/penalty.pkl'
with open('./' + file_name, 'rb') as pen:
     penalty = pickle.load(pen)

In [8]:
rf = Temporal_projection(ref_tr, penalty=penalty, clip_range=(-np.inf, np.inf))

In [9]:
dataset = to_SARS(simulations, rf)